**Import thư viện**

In [ ]:
import numpy as np
import pandas as pd

**Đọc file**

In [ ]:
zipUrl = 'https://cafef1.mediacdn.vn/data/ami_data/20240405/CafeF.SolieuGD.Upto05042024.zip'
import requests
respContent = requests.get(zipUrl, timeout=2).content


In [ ]:
import zipfile
from io import BytesIO

df = None

with zipfile.ZipFile(BytesIO(respContent), 'r') as zf:
    file_names = zf.namelist()

    for file_name in file_names:
        if 'HSX' in file_name:
            with zf.open(file_name, 'r') as f:
                df = pd.read_csv(f)
                break

if df is None:
    print("Không tìm thấy file có chứa 'HSX' trong file zip.")


In [ ]:
print(df)

            <Ticker>  <DTYYYYMMDD>     <Open>     <High>      <Low>  \
0        VNAll-INDEX      20240405  1277.7600  1285.6200  1271.6100   
1                AAA      20240405    11.1000    11.5000    10.9500   
2                AAM      20240405     9.2800     9.2800     9.2700   
3                AAT      20240405     5.1700     5.1800     5.1000   
4                ABR      20240405    12.8500    12.8500    12.8000   
...              ...           ...        ...        ...        ...   
1217734          SAM      20000802     2.0948     2.0948     2.0948   
1217735          REE      20000731     1.1801     1.1801     1.1801   
1217736          SAM      20000731     2.0588     2.0588     2.0588   
1217737          REE      20000728     1.1584     1.1584     1.1584   
1217738          SAM      20000728     2.0349     2.0349     2.0349   

           <Close>   <Volume>  
0        1274.4700  829910000  
1          11.2500    6864600  
2           9.2700        600  
3           5.1300 

**Xem thông tin dữ liệu**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1217739 entries, 0 to 1217738
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   <Ticker>      1217739 non-null  object 
 1   <DTYYYYMMDD>  1217739 non-null  int64  
 2   <Open>        1217739 non-null  float64
 3   <High>        1217739 non-null  float64
 4   <Low>         1217739 non-null  float64
 5   <Close>       1217739 non-null  float64
 6   <Volume>      1217739 non-null  int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 65.0+ MB


In [ ]:
df.isna() .sum ()

<Ticker>        0
<DTYYYYMMDD>    0
<Open>          0
<High>          0
<Low>           0
<Close>         0
<Volume>        0
dtype: int64

**Lọc dữ liệu**

In [ ]:
filtered_df = df[["<Ticker>", "<DTYYYYMMDD>", "<Close>"]]

filtered_df = filtered_df.loc[(df['<Ticker>'].isin(['BMP', 'FPT', 'GAS', 'HPG', 'IMP', 'MSN', 'MWG', 'PTB', 'VCB', 'VNM'])) &
                              (df['<DTYYYYMMDD>'] >= 20190301) &
                              (df['<DTYYYYMMDD>'] <= 20240301)]
filtered_df['<DateTime>'] = pd.to_datetime(filtered_df['<DTYYYYMMDD>'], format='%Y%m%d')
filtered_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 12504 entries, 13822 to 574232
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   <Ticker>      12504 non-null  object        
 1   <DTYYYYMMDD>  12504 non-null  int64         
 2   <Close>       12504 non-null  float64       
 3   <DateTime>    12504 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 488.4+ KB


<ipython-input-7-0abdf258679e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['<DateTime>'] = pd.to_datetime(filtered_df['<DTYYYYMMDD>'], format='%Y%m%d')


**Lấy ngày đầu mỗi tháng**

In [ ]:
def get_first_day_of_month(group):
    return group['<DateTime>'].min()
first_days_of_month = filtered_df.groupby([filtered_df['<DateTime>'].dt.year, filtered_df['<DateTime>'].dt.month]).apply(get_first_day_of_month)
first_days_of_month = first_days_of_month.reset_index(drop=True)
first_days_of_month


0    2019-03-01
1    2019-04-01
2    2019-05-02
3    2019-06-03
4    2019-07-01
        ...    
56   2023-11-01
57   2023-12-01
58   2024-01-02
59   2024-02-01
60   2024-03-01
Length: 61, dtype: datetime64[ns]

**Lọc dữ liệu, chỉ lấy ngày đầu mỗi tháng**

In [ ]:
filtered_df = filtered_df[filtered_df['<DateTime>'].isin(first_days_of_month)]

filtered_df = filtered_df.reset_index(drop=True)

filtered_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609 entries, 0 to 608
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   <Ticker>      609 non-null    object        
 1   <DTYYYYMMDD>  609 non-null    int64         
 2   <Close>       609 non-null    float64       
 3   <DateTime>    609 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 19.2+ KB


In [ ]:
filtered_df

,<Ticker>,<DTYYYYMMDD>,<Close>,<DateTime>
0,BMP,20240301,115.0000,2024-03-01
1,FPT,20240301,110.8000,2024-03-01
2,GAS,20240301,77.6000,2024-03-01
3,HPG,20240301,30.8500,2024-03-01
4,IMP,20240301,66.8000,2024-03-01
...,...,...,...,...
604,MSN,20190301,72.0197,2019-03-01
605,MWG,20190301,42.1414,2019-03-01
606,PTB,20190301,34.0245,2019-03-01
607,VCB,20190301,40.0869,2019-03-01


**Chuyển đổi dataFrame thành dạng pivot**

In [ ]:
pivot_df = filtered_df.pivot(index='<DTYYYYMMDD>', columns='<Ticker>', values='<Close>')
pivot_df.columns.name = None
print(pivot_df.to_string())


                   BMP       FPT      GAS      HPG      IMP       MSN      MWG      PTB      VCB      VNM
<DTYYYYMMDD>                                                                                             
20190301       32.8921   19.2421  69.3366  11.9920  40.1019   72.0197  42.1414  34.0245  40.0869  92.4784
20190401       30.2729   19.7598  71.2178  11.1867  39.7928   71.2105  40.5039  33.9709  43.6960  90.6471
20190502       28.4456   21.5071  80.6239  11.9045  39.4837   71.6151  41.4189  34.8674  43.1160  85.0225
20190603       28.4456   21.0681  71.7129  11.0292  38.8656   68.7829  41.6411  35.5444  42.8582  83.7145
20190701       27.6635   22.8117  73.3395  10.4694  37.5875   69.2684  46.3930  39.7759  45.9517  82.7203
20190801       31.3306   23.4171  76.5927  10.2873  39.2777   64.8178  53.1536  38.0833  51.8809  82.1909
20190903       31.7166   26.5066  72.2059   9.8549  38.6338   59.8816  57.6116  38.5347  50.1408  82.0585
20191001       33.2606   28.8759  76.8364   9.

**Xử lý giá trị NaN, đặt lại index, đổi tên cột <'DTYYYYMMDD'> thành Date**

In [ ]:
pivot_df.loc[20230103, 'IMP'] = df.loc[(df['<Ticker>'] == 'IMP') & (df['<DTYYYYMMDD>'] == 20230104), '<Close>'].values[0]
pivot_df.reset_index(inplace=True)
pivot_df.rename(columns={'<DTYYYYMMDD>': 'Date'}, inplace=True)
print(pivot_df.to_string())


        Date       BMP       FPT      GAS      HPG      IMP       MSN      MWG      PTB      VCB      VNM
0   20190301   32.8921   19.2421  69.3366  11.9920  40.1019   72.0197  42.1414  34.0245  40.0869  92.4784
1   20190401   30.2729   19.7598  71.2178  11.1867  39.7928   71.2105  40.5039  33.9709  43.6960  90.6471
2   20190502   28.4456   21.5071  80.6239  11.9045  39.4837   71.6151  41.4189  34.8674  43.1160  85.0225
3   20190603   28.4456   21.0681  71.7129  11.0292  38.8656   68.7829  41.6411  35.5444  42.8582  83.7145
4   20190701   27.6635   22.8117  73.3395  10.4694  37.5875   69.2684  46.3930  39.7759  45.9517  82.7203
5   20190801   31.3306   23.4171  76.5927  10.2873  39.2777   64.8178  53.1536  38.0833  51.8809  82.1909
6   20190903   31.7166   26.5066  72.2059   9.8549  38.6338   59.8816  57.6116  38.5347  50.1408  82.0585
7   20191001   33.2606   28.8759  76.8364   9.9687  39.4387   63.7658  62.4616  40.1709  53.7499  87.5538
8   20191101   34.0969   28.8759  75.6064   9.

**Tổ hợp chập 3 của 10, lấy 120 bảng, mỗi bảng gồm 3 cổ phiếu**

In [ ]:
import itertools

stock_symbols = ['BMP', 'FPT', 'GAS', 'HPG', 'IMP', 'MSN', 'MWG', 'PTB', 'VCB', 'VNM']
stock_combinations = list(itertools.combinations(stock_symbols, 3))
tables = []

for i in range(len(stock_combinations)):
    stock_comb = stock_combinations[i]
    table = pd.DataFrame({'Date': pivot_df['Date'],
                          stock_comb[0]: pivot_df[stock_comb[0]],
                          stock_comb[1]: pivot_df[stock_comb[1]],
                          stock_comb[2]: pivot_df[stock_comb[2]]})
    tables.append(table)

for i, table in enumerate(tables):
    print(f'Bảng {i+1}:')
    print(table)
    print()


Bảng 1:
        Date       BMP       FPT      GAS
0   20190301   32.8921   19.2421  69.3366
1   20190401   30.2729   19.7598  71.2178
2   20190502   28.4456   21.5071  80.6239
3   20190603   28.4456   21.0681  71.7129
4   20190701   27.6635   22.8117  73.3395
..       ...       ...       ...      ...
56  20231101   75.0502   85.4000  76.9000
57  20231201   87.2000   92.4000  77.7000
58  20240102  109.0000   95.9000  75.6000
59  20240201  107.1000   99.9000  75.5000
60  20240301  115.0000  110.8000  77.6000

[61 rows x 4 columns]

Bảng 2:
        Date       BMP       FPT      HPG
0   20190301   32.8921   19.2421  11.9920
1   20190401   30.2729   19.7598  11.1867
2   20190502   28.4456   21.5071  11.9045
3   20190603   28.4456   21.0681  11.0292
4   20190701   27.6635   22.8117  10.4694
..       ...       ...       ...      ...
56  20231101   75.0502   85.4000  23.9500
57  20231201   87.2000   92.4000  26.9000
58  20240102  109.0000   95.9000  27.4500
59  20240201  107.1000   99.9000  27

In ra 1 bảng kiểm tra

In [ ]:
print(tables[118].to_string())

        Date      MWG      VCB      VNM
0   20190301  42.1414  40.0869  92.4784
1   20190401  40.5039  43.6960  90.6471
2   20190502  41.4189  43.1160  85.0225
3   20190603  41.6411  42.8582  83.7145
4   20190701  46.3930  45.9517  82.7203
5   20190801  53.1536  51.8809  82.1909
6   20190903  57.6116  50.1408  82.0585
7   20191001  62.4616  53.7499  87.5538
8   20191101  61.3348  57.2301  90.5634
9   20191202  53.1046  53.8144  80.9188
10  20200102  57.3177  59.0387  81.3366
11  20200203  53.3006  57.6732  74.7138
12  20200302  52.1738  53.1868  72.0922
13  20200401  29.9816  42.2634  64.8485
14  20200504  38.8977  43.8238  67.3321
15  20200601  42.1310  56.2428  80.0948
16  20200701  40.1714  53.7720  80.1631
17  20200803  37.2320  51.2362  76.3192
18  20200901  45.9521  53.9671  86.1037
19  20201001  51.4390  55.1374  93.1233
20  20201102  52.8542  55.7876  91.6749
21  20201201  57.6230  59.8189  93.4641
22  20210104  59.9081  64.8960  93.1233
23  20210201  61.4480  59.7174  88.3126


**Establish the optimal risky portfolio**

In [ ]:
rf = (7/100)/12
print(rf)

0.005833333333333334


**Maximize Sharpe Ratio**

In [ ]:
from scipy.optimize import minimize
def sharpe_ratio(W, mean1, mean2, mean3, var1, var2, var3, cov12, cov13, cov23):
  Erp = W[0] * mean1 + W[1] * mean2+ W[2] *mean3
  op = math.sqrt(W[0]**2 * var1 + W[1]**2 * var2 + W[2]**2 * var3 +
                    2 * W[0] * W[1] * cov12 + 2 * W[0] * W[2] * cov13 + 2 * W[1] * W[2] * cov23)
  sharpe_ratio = (Erp - rf) / op
  return -sharpe_ratio
def constraint(W):
  return sum(W) - 1


In [ ]:
import math
means = []
vars = []
stds = []
covs = []
corrs = []
Ws = []
Erps = []
ops = []
sharpeRatios = []
for i, table in enumerate(tables):
  table['Return1'] = (table.iloc[:, 1] / table.iloc[:, 1].shift(1)) - 1
  table['Return2'] = (table.iloc[:, 2] / table.iloc[:, 2].shift(1)) - 1
  table['Return3'] = (table.iloc[:, 3] / table.iloc[:, 3].shift(1)) - 1
  mean1 = table['Return1'].mean()
  mean2 = table['Return2'].mean()
  mean3 = table['Return3'].mean()
  table['Return1 - mean'] = table['Return1'] - mean1
  table['Return2 - mean'] = table['Return2'] - mean2
  table['Return3 - mean'] = table['Return3'] - mean3
  table['(Return1 - mean)(Return2 - mean)'] = table['Return1 - mean']*table['Return2 - mean']
  table['(Return1 - mean)(Return3 - mean)'] = table['Return1 - mean']*table['Return3 - mean']
  table['(Return2 - mean)(Return3 - mean)'] = table['Return2 - mean']*table['Return3 - mean']
  table['(Return1 - mean)^2'] = table['Return1 - mean']**2
  table['(Return2 - mean)^2'] = table['Return2 - mean']**2
  table['(Return3 - mean)^2'] = table['Return3 - mean']**2

  var1 = table['(Return1 - mean)^2'].sum()/(len(table)-1)
  var2 = table['(Return2 - mean)^2'].sum()/(len(table)-1)
  var3 = table['(Return3 - mean)^2'].sum()/(len(table)-1)
  std1 = math.sqrt(var1)
  std2 = math.sqrt(var2)
  std3 = math.sqrt(var3)
  cov12 = table['(Return1 - mean)(Return2 - mean)'].mean()
  cov13 = table['(Return1 - mean)(Return3 - mean)'].mean()
  cov23 = table['(Return2 - mean)(Return3 - mean)'].mean()
  corr12 = table['Return1'].corr(table['Return2'])
  corr13 = table['Return1'].corr(table['Return3'])
  corr23 = table['Return2'].corr(table['Return3'])
  initial_guess = [0.25, 0.25, 0.5]
  bounds = ((0.05, 1), (0.05, 1), (0.05, 1))
  result = minimize(sharpe_ratio, initial_guess, args=(mean1, mean2, mean3, var1, var2, var3, cov12, cov13, cov23),
                    method='SLSQP', bounds=bounds, constraints={'type': 'eq', 'fun': constraint},
                    options={'ftol': 1e-9, 'disp': True})

  optimal_weights = result.x
  optimal_sharpe_ratio = -result.fun
  W1= optimal_weights[0]
  W2= optimal_weights[1]
  W3 = optimal_weights[2]
  Erp = W1*mean1+W2*mean2+W3*mean3
  op = math.sqrt(W1**2 * std1**2+W2**2 * std2**2+ W3**2*std3**2
                 +2*W1*W2*cov12+2*W1*W3*cov13+2*W2*W3*cov23)
  sharpeRatio = (Erp-rf)/op

  means.append([mean1, mean2, mean3])
  vars.append([var1, var2, var3])
  stds.append([std1, std2, std3])
  covs.append([cov12, cov13, cov23])
  corrs.append([corr12, corr13, corr23])
  Ws.append([W1, W2, W3])
  Erps.append(Erp)
  ops.append(op)
  sharpeRatios.append(sharpeRatio)


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.33207789208134814
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.34073916094799106
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.3409835583348132
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.334189016958392
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.32589636264693206
            Iterations: 6
            Function evalu

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.3447552456729252
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.3311388900526334
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.2094337945206379
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.18824662195011438
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.18659436710397123
            Iterations: 3
            Function evalu

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


...

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.15409716347196542
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.08651633862678997
            Iterations: 5
            Function evaluations: 20
            Gradient evaluations: 5


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.1424984075269125
            Iterations: 4
            Function evaluations: 16
            Gradient evaluations: 4
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.1540387955424553
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.08687202326419648
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.1432669489876804
            Iterations: 5
            Function evaluations: 20
            Gradient evaluations: 5
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.14694180941969695
            Iterations: 7
            Function evalu

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


**In 1 bảng để xem dữ liệu**



In [ ]:
#In 1 bảng để xem dữ liệu
print(tables[118].to_string())
print('rf: ', rf)
print('mean1: ', means[118][0])
print('mean2: ', means[118][1])
print('mean3: ', means[118][2])
print('std1: ', stds[118][0])
print('std2: ', stds[118][1])
print('std3: ', stds[118][2])
print('var1: ', vars[118][0])
print('var2: ', vars[118][1])
print('var3: ', vars[118][2])
print('cov12: ', covs[118][0])
print('cov13: ', covs[118][1])
print('cov23: ', covs[118][2])
print('corr12: ', corrs[118][0])
print('corr13: ', corrs[118][1])
print('corr23: ', corrs[118][2])
print('E(rp): ', Erps[118])
print('o(p) : ', ops[118])
print('SharpeRatio: ', sharpeRatios[118])

        Date      MWG      VCB      VNM   Return1   Return2   Return3  Return1 - mean  Return2 - mean  Return3 - mean  (Return1 - mean)(Return2 - mean)  (Return1 - mean)(Return3 - mean)  (Return2 - mean)(Return3 - mean)  (Return1 - mean)^2  (Return2 - mean)^2  (Return3 - mean)^2
0   20190301  42.1414  40.0869  92.4784       NaN       NaN       NaN             NaN             NaN             NaN                               NaN                               NaN                               NaN                 NaN                 NaN                 NaN
1   20190401  40.5039  43.6960  90.6471 -0.038857  0.090032 -0.019802       -0.049773        0.072082       -0.017019                         -0.003588                          0.000847                         -0.001227            0.002477            0.005196        2.896557e-04
2   20190502  41.4189  43.1160  85.0225  0.022590 -0.013274 -0.062049        0.011674       -0.031223       -0.059266                         -0.000365         

**MaxSp**

In [ ]:

maxSharpeRatio = max(sharpeRatios)
print("Giá trị Sharpe Ratio lớn nhất từ 120 bảng là:", maxSharpeRatio)
max_index = sharpeRatios.index(maxSharpeRatio)
print("Bảng đó là bảng thứ:", max_index+1)


Giá trị Sharpe Ratio lớn nhất từ 120 bảng là: 0.3447552456729252
Bảng đó là bảng thứ: 7


**In thông tin bảng có SharpeRatio max**

In [ ]:
print(tables[max_index].to_string())
print('rf: ', rf)
print('mean1: ', means[max_index][0])
print('mean2: ', means[max_index][1])
print('mean3: ', means[max_index][2])
print('std1: ', stds[max_index][0])
print('std2: ', stds[max_index][1])
print('std3: ', stds[max_index][2])
print('var1: ', vars[max_index][0])
print('var2: ', vars[max_index][1])
print('var3: ', vars[max_index][2])
print('cov12: ', covs[max_index][0])
print('cov13: ', covs[max_index][1])
print('cov23: ', covs[max_index][2])
print('corr12: ', corrs[max_index][0])
print('corr13: ', corrs[max_index][1])
print('corr23: ', corrs[max_index][2])
print('E(rp): ', Erps[max_index])
print('o(p) : ', ops[max_index])
print('SharpeRatio: {0}%'.format(round(sharpeRatios[max_index]*100, 2)))

        Date       BMP       FPT      VCB   Return1   Return2   Return3  Return1 - mean  Return2 - mean  Return3 - mean  (Return1 - mean)(Return2 - mean)  (Return1 - mean)(Return3 - mean)  (Return2 - mean)(Return3 - mean)  (Return1 - mean)^2  (Return2 - mean)^2  (Return3 - mean)^2
0   20190301   32.8921   19.2421  40.0869       NaN       NaN       NaN             NaN             NaN             NaN                               NaN                               NaN                               NaN                 NaN                 NaN                 NaN
1   20190401   30.2729   19.7598  43.6960 -0.079630  0.026905  0.090032       -0.105899       -0.006104        0.072082                          0.000646                         -0.007633                         -0.000440            0.011215            0.000037            0.005196
2   20190502   28.4456   21.5071  43.1160 -0.060361  0.088427 -0.013274       -0.086630        0.055419       -0.031223                         -0.004801 

**Establish the optimal complete portfolio**

In [ ]:
A = 25
y = (Erps[max_index]-rf)/(A*(ops[max_index]**2))
Erc = y*Erps[max_index]+(1-y)*rf
oc = y*ops[max_index]
U = Erc-0.5*A*(oc**2)
print('U: {0}%'.format(round(U*100, 2)))

U: 0.82%
